### 1. Image Augmentation

https://hoya012.github.io/blog/albumentation_tutorial/
<br>해당 페이지를 참고했을 때, imgaug 방법 보다는 albumentations를 사용하는 것이 img augmentation에 적절하다는 판단

In [1]:
#pip install albumentations

  Using cached scipy-1.13.1-cp39-cp39-macosx_10_9_x86_64.whl.metadata (60 kB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached pillow-10.4.0-cp39-cp39-macosx_10_10_x86_64.whl.metadata (9.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 10.8 MB/s eta 0:00:0000:0100:01
Using cached scipy-1.13.1-cp39-cp39-macosx_10_9_x86_64.whl (39.4 MB)
Using cached networkx-3.2.1-py3-none-any.whl (1.6 MB)
Using cached pillow-10.4.0-cp39-cp39-macosx_10_10_x86_64.whl (3.5 MB)
  Attempting uninstall: tifffile
    Found existing installation: tifffile 2021.7.2
    Uninstalling tifffile-2021.7.2:
      Successfully uninstalled tifffile-2021.7.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.1
    Uninstalling scipy-1.7.1:
      Successfully uninstalled scipy-1.7.1
  Attempting

In [1]:
import cv2 as cv
import numpy as np
import os
import glob
import albumentations as albu

In [2]:
# Augmentation pipeline 정의 (회전, 뒤집기, 기울기 등 적용)
# 여기서 밝기, 색조 등 색에 영향을 끼칠 만한 요소는 제외하고 augmentation 진행

augmentation_pipeline = albu.Compose([
    albu.HorizontalFlip(p=0.5), # 50% 확률로 좌우 반전
    albu.RandomRotate90(p=0.5), # 90도 회전
    albu.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, p=0.5), # 이미지 이동, 크기 조절, 회전
    albu.GaussNoise(p=0.2)  # 가우시안 노이즈 추가
])

In [3]:
def augment_images(image_set, output_path, num_augmented):
    # 출력할 img folder가 없으면 생성~
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    for i, img_path in enumerate(image_set):
        # 이미지 불러오기
        img = cv.imread(img_path)   #! img_path가 어딨농?

        if img is None:
            print(f"이미지 불러오기 실패!!! : {img_path}")
            continue
        
        for j in range(num_augmented):
            # augmentation 적용
            augmented = augmentation_pipeline(image=img)
            augmented_img = augmented['image']

            # 증강된 이미지 저장 경로 설정 및 저장
            output_file = os.path.join(output_path, f"augmented_{i}_{j}.jpg")
            cv.imwrite(output_file, augmented_img)

In [4]:
# image set augmentation

# 증강할 이미지 폴더 경로 설정
image_folder = '/Users/hyunowk/Downloads/aiclopse_hyun/img/*'

# glob을 이용해 이미지 경로 리스트 생성
image_paths = glob.glob(image_folder)

In [5]:
# image_folder 내 이미지들 각각 20번씩 증강하여 저장
augment_images(image_paths, 'output_folder', 20)

### 2. 이미지 전처리 (Grayscale 및 mask 추출)

In [11]:
def process_images(input_folder):
    # 처리된 이미지를 저장할 폴더가 없으면 생성
    if not os.path.exists("processed_images"):
        os.makedirs("processed_images")
    
    if not os.path.exists("mask_images"):
        os.makedirs("mask_images")
    
    images = os.listdir(input_folder)

    for img_file in images:
        img_path = os.path.join(input_folder, img_file)
        img = cv.imread(img_path)

        if img is None:
            print(f"이미지를 불러올 수 없습니다: {img_path}")
            continue

        # 1. HSV 변환
        img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

        # 2. 보라색 범위 설정 (이전에 구해 놓은 범위로 파란색, 보라색 범위 구분해서 설정)
        # blue 범위
        lower_blue = np.array([100, 150, 0])
        upper_blue = np.array([140, 255, 255])

        # purple 범위
        lower_purple = np.array([125, 50, 70])
        upper_purple = np.array([150, 255, 255])

        # 3. 보라색 mask 생성
        blue_mask = cv.inRange(img_hsv, lower_blue, upper_blue)
        purple_mask = cv.inRange(img_hsv, lower_purple, upper_purple)

        # 4. 각 이미지가 보라색에 해당하면 보라색 마스크, 파란색에 해당하면 파란색 마스크 생성
        if cv.countNonZero(purple_mask) > 0:
            # 보라색 마스크가 있는 경우
            mask = purple_mask
            mask_color = "purple"
        elif cv.countNonZero(blue_mask) > 0:
            # 파란색 마스크가 있는 경우
            mask = blue_mask
            mask_color = "blue"
        else:
            print(f"{img_file}에서 파란색 또는 보라색이 감지되지 않았습니다.")
            continue

        # 5. 흑백 변환 (보라색 Mask 적용 후)
        img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        
        # 6. mask 적용된 흑백 이미지 저장
        processed_img_path = os.path.join("processed_images", img_file)
        cv.imwrite(processed_img_path, img_gray)

        # 7. 해당 마스크 저장 (보라색 또는 파란색)
        mask_img_path = os.path.join("mask_images", f"{mask_color}_mask_{img_file}")
        cv.imwrite(mask_img_path, mask)

        print(f"{img_file}의 {mask_color} 마스크 및 처리된 이미지 저장 완료")

In [12]:
# 증강된 이미지들 폴더 경로에서 처리
process_images('output_folder')

augmented_8_10.jpg의 purple 마스크 및 처리된 이미지 저장 완료
augmented_10_15.jpg의 purple 마스크 및 처리된 이미지 저장 완료
augmented_6_15.jpg의 purple 마스크 및 처리된 이미지 저장 완료
augmented_13_8.jpg의 purple 마스크 및 처리된 이미지 저장 완료
augmented_12_10.jpg의 purple 마스크 및 처리된 이미지 저장 완료
augmented_4_10.jpg의 purple 마스크 및 처리된 이미지 저장 완료
augmented_3_19.jpg의 purple 마스크 및 처리된 이미지 저장 완료
augmented_6_9.jpg의 purple 마스크 및 처리된 이미지 저장 완료
augmented_8_6.jpg의 purple 마스크 및 처리된 이미지 저장 완료
augmented_8_7.jpg의 purple 마스크 및 처리된 이미지 저장 완료
augmented_6_8.jpg의 purple 마스크 및 처리된 이미지 저장 완료
augmented_4_11.jpg의 purple 마스크 및 처리된 이미지 저장 완료
augmented_3_18.jpg의 purple 마스크 및 처리된 이미지 저장 완료
augmented_12_11.jpg의 purple 마스크 및 처리된 이미지 저장 완료
augmented_13_9.jpg의 purple 마스크 및 처리된 이미지 저장 완료
augmented_6_14.jpg의 purple 마스크 및 처리된 이미지 저장 완료
augmented_10_14.jpg의 purple 마스크 및 처리된 이미지 저장 완료
augmented_8_11.jpg의 purple 마스크 및 처리된 이미지 저장 완료
augmented_10_16.jpg의 purple 마스크 및 처리된 이미지 저장 완료
augmented_8_13.jpg의 purple 마스크 및 처리된 이미지 저장 완료
augmented_6_16.jpg의 purple 마스크 및 처리된 이미지 저장 완료
augmented_4_

### 3. Threshold 및 Training

In [15]:
def find_threshold(mask_images):

    best_threshold = 0

    # 여러 이미지들에서 최적의 threshold 값을 찾아나가자-! (평균 계산으로 1차적으로 간단히 해봄)
    thresholds = []
    
    for mask_img in mask_images:

        if mask_img is None:
            print("이미지를 불러올 수 없습니다..")
            continue
        
        # 적절한 threshold 값 찾깅ㅋ (otsu 방식 적용)
        _, thresholded_img = cv.threshold(mask_img, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
        thresholds.append(thresholded_img)
    
    # 전체 이미지의 평균 threshold값 구하기
    best_threshold = np.mean(thresholds)    # 평균
    return best_threshold

In [16]:
# mask 이미지 경로들
mask_images = [
    cv.imread('mask1.jpg', cv.IMREAD_GRAYSCALE), 
    cv.imread('mask2.jpg', cv.IMREAD_GRAYSCALE)
    ]

# 최적의 threshold 값 찾기
best_threshold = find_threshold(mask_images)

if best_threshold is not None:
    print("Best threshold: ", best_threshold)
else:
    print("Threshold 값을 계산할 수 없습니다.")

이미지를 불러올 수 없습니다..
이미지를 불러올 수 없습니다..
Best threshold:  nan


[ WARN:0@950.118] global loadsave.cpp:241 findDecoder imread_('mask1.jpg'): can't open/read file: check file path/integrity
[ WARN:0@950.120] global loadsave.cpp:241 findDecoder imread_('mask2.jpg'): can't open/read file: check file path/integrity
/Users/hyunowk/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/hyunowk/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


### 4. 새로운 입력 이미지에 적용해보기

In [ ]:
def detect_circle_in_new_image(new_image, threshold):
    hsv_image = cv.cvtColor(new_image, cv.COLOR_BGR2HSV)
    lower_purple = np.array([125, 50, 50])
    upper_purple = np.array([150, 255, 255])
    purple_mask = cv.inRange(hsv_image, lower_purple, upper_purple)

    # 학습된 threshold를 적용
    _, thresholded_img = cv.threshold(purple_mask, threshold, 255, cv.THRESH_BINARY)
    
    # 원 검출
    circles = cv.HoughCircles(thresholded_img, cv.HOUGH_GRADIENT, dp=1.2, minDist=30,
                               param1=50, param2=30, minRadius=10, maxRadius=100)
    
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            cv.circle(new_image, (x, y), r, (0, 255, 0), 4)

    cv.imshow('Detected Circles', new_image)
    cv.waitKey(0)
    cv.destroyAllWindows()

# 새로운 이미지에 대해 학습된 threshold 적용
new_img = cv.imread('new_laser_image.jpg')
detect_circle_in_new_image(new_img, best_threshold)
